In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [ ]:
train_df=pd.read_csv("https://raw.githubusercontent.com/Enthusiasticguy/AirticketPrediction/refs/heads/main/train_data.csv")
train_df.head()

,id,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,1,Vistara,UK-810,Bangalore,Early_Morning,one,Night,Mumbai,Economy,14.25,21,7212
1,2,SpiceJet,SG-5094,Hyderabad,Evening,zero,Night,Kolkata,Economy,1.75,7,5292
2,3,Vistara,UK-846,Bangalore,Morning,one,Evening,Delhi,Business,9.58,5,60553
3,4,Vistara,UK-706,Kolkata,Morning,one,Evening,Hyderabad,Economy,6.75,28,5760
4,5,Indigo,6E-5394,Chennai,Early_Morning,zero,Morning,Mumbai,Economy,2.00,4,10712



*   5 xil `airline` bor bizda - bularga One hot encoder
*   `flight` larga shunchaki - nomlari hech qanaqa narsani anglatmagani uchun simple encoder
*   6 xil `source_city` - bularga One hot encoder
*   6 xil `departure time` - bularga One hot encoder
*   `stops` - bunga Ordinal encoder ishlatamiz
*  ` arrival_time` - bunga ham OnehotEncoder
*   `destination_city` - bunga ham One hot encoder
*   `class` - ga ham Onehotencoder
*   `duration` - Standard scaler
*   `days_left` - Standard scaler



`price` , bu bizning label imiz bo'ladi




In [ ]:
train_set, test_set = train_test_split(train_df, test_size=0.2, random_state=95)

X_train=train_set.drop(['price'],axis=1)
y_train=train_set['price']

X_test=test_set.drop(['price'],axis=1)
y_test=test_set['price']

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor

# ustunlar
one_hot_columns = ['airline', 'source_city', 'departure_time', 'arrival_time', 'destination_city', 'class']
ordinal_columns = ['flight', 'stops']
numerical_columns = ['duration', 'days_left']

# Preprocessing raqamli malumotlar uchun (StandardScaler)
num_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Preprocessing satrli malumotlar uchun OneHotEncoder
cat_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Preprocessing kategoriyali malumotlar uchun OrdinalEncoder
ordinal_transformer = Pipeline(steps=[
    ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

#  ColumnTransformer orqali, shablon yaratamiz va yigamiz
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, numerical_columns),
        ('cat', cat_transformer, one_hot_columns),
        ('ord', ordinal_transformer, ordinal_columns)
    ]
)

# Modelni ham qo'shib olsak
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor())])



In [ ]:
#Modelni oqitish jarayonidamiz
model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['duration', 'days_left']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['airline', 'source_city',
                                                   'departure_time',
                                                   'arrival_time',
                                                   'destination_city',
                                                   'class']),
                                                 ('ord',
                                                  Pipeline(steps=[('ordinal',
                                                                   OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                  unknown_value=-1))]),
                                                  ['flight', 'stops'])])),
                ('model', RandomForestRegressor())])

In [ ]:
#Accuarcy larni o'lchaymiz
from sklearn.metrics import mean_squared_error

#  Bahsorat qilib oldik
y_pred = model_pipeline.predict(X_test)

#  RMSE score
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse}")

RMSE: 3148.892891875487
